# SQUID SMILE interpolation of GPS location data

Emilio, 8/1/2023

Apply GPS latitude and longitude data to the `vel` data file based on interpolation on timestamps.

In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd

In [2]:
base_dpth = Path("/Users/lily/Documents/Elizabeth's Files 2023/DINO SIP Documents/")

In [3]:
gps_parquet_path = "gps_points_allexperiments.parquet"
ctd_parquet_path = "ctd_points_allexperiments.parquet"
vel_parquet_path = "vel_points_allexperiments.parquet"

## Read the data (parquet files)

For the GPS file, let's load only the columsn we're interested in. That will make processing faster, as it'll consume less memory

In [4]:
gps_df = pd.read_parquet(base_dpth / gps_parquet_path)[['UXT_DT', 'LAT', 'LON', 'deployment', 'experiment']]
vel_df = pd.read_parquet(base_dpth / vel_parquet_path)

I find it helpful to examine the DataFrames a bit, for reference

In [5]:
len(gps_df), len(vel_df)

(591822, 2619890)

In [6]:
gps_df.columns

Index(['UXT_DT', 'LAT', 'LON', 'deployment', 'experiment'], dtype='object')

In [7]:
vel_df.columns

Index(['deployment', 'experiment', 'dive_nbr', 'P', 'T', 'S', 'u1', 'v1',
       'verr1', 'u2', 'v2', 'verr2', 'W', 'e1mean', 'e2mean', 'piston', 'uxt',
       'UXT_DT'],
      dtype='object')

## Merge the two DataFrames

This merging step will include columns and rows from both DataFrames. It'll have lots of empty cells ("NaN", Not-A-Number). Before merging, we filter the two DataFrames to retain only SMILE records.

In [8]:
vel_gps_df = (
    gps_df[gps_df.experiment == 'SMILE']
    .merge(vel_df[vel_df.experiment == 'SMILE'], how='outer')
    .set_index('UXT_DT')
)

In [9]:
len(vel_gps_df)

631603

In [10]:
vel_gps_df.head()

,LAT,LON,deployment,experiment,dive_nbr,P,T,S,u1,v1,verr1,u2,v2,verr2,W,e1mean,e2mean,piston,uxt
UXT_DT,,,,,,,,,,,,,,,,,,,
2017-03-08 05:53:59+00:00,26.238247,-146.275868,4968s1,SMILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-08 05:53:59+00:00,26.238247,-146.275868,4968s1,SMILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-08 05:54:00+00:00,26.238247,-146.275868,4968s1,SMILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-08 05:54:00+00:00,26.238247,-146.275868,4968s1,SMILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-03-08 05:54:01+00:00,26.238247,-146.275868,4968s1,SMILE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Interpolate latitude and longitude values on the timestamp values

Every timestamp from the `vel` DataFrame will be assigned a latitude and longitude value based on interpolation of latitudes and longitudes over the GPS timestamps.

A different interpolation method could be used, but linear seems good enough for our purposes.

In [11]:
vel_gps_df = vel_gps_df.interpolate(method='slinear').dropna(subset="v1")

ValueError: Expect x to not have duplicates

In [ ]:
len(vel_gps_df)

In [ ]:
vel_gps_df.head()

Now let's export it to a parquet file

In [ ]:
vel_gps_df.to_parquet(base_dpth / 'vel_gps_interpolated.parquet')

## Let's look at the issue of data from 2019

We can examine this via a histogram. `uxt` is a numeric timestmap column (seconds since 1970-01-01). As we can see, A small fraction of the data have bad timestamps, from 2019. It'd be better to go back to earlier steps and filter it out based on, say, `gps_df.UXT_DT > '2018-01-01'` and `vel_df.UXT_DT > '2018-01-01'

In [ ]:
vel_gps_df.index.min(), vel_gps_df.index.max()

In [ ]:
vel_gps_df.uxt.hist();